In [ ]:
!pip3 install newspaper3k
!pip3 install feedparser

     |████████████████████████████████| 215kB 5.9MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 7.4MB 38.1MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=46ef1d3d55d44207521e06dde9b2f1bff8daf87e8d35a06b0d04ad3bd432e22f
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3358 sha256=8868acf1d1e657788f2a8fc387de2d5a9652bb22263477fec9e251cfe93a83be
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=c37db536e757f69e143eb0687e5819d6a7e394620aed37a0cde78cdf0a3ac0ed
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35

In [ ]:
import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

# create folder to write data to
Fake_News=os.path.join(DRIVE_MOUNT, 'My Drive', 'Fake News')
Project=os.path.join(Fake_News, 'Project')
os.makedirs(Project, exist_ok=True)

Mounted at /content/gdrive


In [ ]:
import json
dictionary = {
  "cnn": {
    "link": "http://edition.cnn.com/"
  },
  "bbc": {
    "rss": "http://feeds.bbci.co.uk/news/rss.xml",
    "link": "http://www.bbc.com/"
  },
  "theguardian": {
    "rss": "https://www.theguardian.com/uk/rss",
    "link": "https://www.theguardian.com/international"
  },
  "breitbart": {
    "link": "http://www.breitbart.com/"
  },
  "infowars": {
    "link": "https://www.infowars.com/"
  },
  "foxnews": {
    "link": "http://www.foxnews.com/"
  },
  "nbcnews": {
    "link": "http://www.nbcnews.com/"
  },
  "washingtonpost": {
    "rss": "http://feeds.washingtonpost.com/rss/world",
    "link": "https://www.washingtonpost.com/"
  },
  "theonion": {
    "link": "http://www.theonion.com/"
  },
  "sky": {
    "link": "http://news.sky.com/"
  },
  "msnbc": {
    "link": "http://www.msnbc.com/"
  },
  "NDTV": {
    "link": "http://www.ndtv.com/india"
  },
  "worldnewsdailyreport": {
    "link": "https://worldnewsdailyreport.com/"
  },
  "prntly": {
    "link": "https://prntly.com/"
}
}

In [ ]:
json_object = json.dumps(dictionary, indent = 4)
with open("NewsPapers.json", "w") as outfile: 
    outfile.write(json_object)

In [ ]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

# Set the limit for number of articles to download
LIMIT = 14500

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 100:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

# Finally it saves the articles as a JSON-file.
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

Building site for  cnn
1  Article has date of type None...
2  Article has date of type None...
3 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2021/02/11/media/disney-earnings-q1-2021/index.html
4 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2021/02/11/media/bbc-news-banned-china/index.html
5 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2021/02/11/investing/elon-musk-pay-wealth-tesla-stock-options/index.html
6 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2021/02/11/success/buying-tesla-with-bitcoin-feseries/index.html
7 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2021/02/11/investing/robinhood-lawsuit-suicide-alex-kearns/index.html
8 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2021/02/11/investing/mastercard-bitcoin-bny-mellon/index.html
9 articles downloaded from cnn  using newspaper, url:  http://edi

In [ ]:
with open('scraped_articles.json') as json_data:
    d = json.load(json_data)

In [ ]:
for i, site in enumerate((list(d['newspapers']))):
    print(i, site)

0 cnn
1 bbc
2 theguardian
3 breitbart
4 infowars
5 foxnews
6 nbcnews
7 washingtonpost
8 theonion
9 sky
10 msnbc
11 NDTV
12 worldnewsdailyreport
13 prntly


In [ ]:

import pandas as pd
for i, site in enumerate((list(d['newspapers']))):
    articles = list(d['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
        df["site"] = site
    else:
        new_df = pd.DataFrame.from_dict(articles)
        new_df["site"] = site
        df = pd.concat([df, new_df], ignore_index = True)

In [ ]:
df.shape

(1911, 5)

In [ ]:
df.head()

,title,text,link,published,site
0,Disney is still getting hammered by the pandem...,New York (CNN Business) Disney kicked off its ...,http://edition.cnn.com/2021/02/11/media/disney...,2021-02-11T00:00:00,cnn
1,"BBC News banned in China, one week after CGTN'...",Atlanta (CNN Business) In an apparent tit for ...,http://edition.cnn.com/2021/02/11/media/bbc-ne...,2021-02-11T00:00:00,cnn
2,"Elon Musk, the world's richest man, is about t...",He received four grants to buy 8.4 million Tes...,http://edition.cnn.com/2021/02/11/investing/el...,2021-02-11T00:00:00,cnn
3,How risky is it to buy a Tesla using bitcoin?,(CNN) Tesla revealed in a securities filing th...,http://edition.cnn.com/2021/02/11/success/buyi...,2021-02-11T00:00:00,cnn
4,'He would be alive today': Parents detail son'...,New York (CNN Business) After their son's deat...,http://edition.cnn.com/2021/02/11/investing/ro...,2021-02-11T00:00:00,cnn


In [ ]:
!cp scraped_articles.csv "/content/gdrive/My Drive/Fake News/Project/"

In [ ]:
df.to_csv('scraped_articles.csv')
